# 1. 作用

`tf.keras.models.load_model()` 用于从磁盘加载一个保存的 `Keras` 模型（通过 `model.save()` 保存的），恢复：

 - 模型结构（层、连接关系等）

 - 权重参数

 - 训练配置（如 `loss`、`optimizer`、`metrics` 等）

加载后的模型与原始模型完全等效，可以直接用于预测、评估或继续训练。

---
# 2. 基本语法和用法

In [ ]:
from tensorflow.keras.models import load_model

model = tf.keras.models.load_model(
    filepath,
    custom_objects=None,
    compile=True,
    options=None
)

| 参数                   | 说明                                                            |
| -------------------- | ------------------------------------------------------------- |
| **`filepath`**       | 模型文件路径，可以是 `.h5` 文件或 `SavedModel` 目录                          |
| **`custom_objects`** | 字典，若模型中有自定义层/激活函数/损失函数，需要在这里传入，例如：`{'MyLayer': MyLayer}`      |
| **`compile`**        | 是否加载训练配置（optimizer、loss、metrics）。默认 `True`，如果只是推理可以设为 `False` |
| **`options`**        | 可选的 `tf.saved_model.LoadOptions` 参数，用于控制 SavedModel 加载行为      |


---

# 3. 完整工作流程示例
让我们通过一个完整的例子来演示保存和加载的流程。

In [ ]:
import tensorflow as tf
import numpy as np

# 1. 创建、编译并训练一个简单的模型
print("1. 创建并训练原始模型...")
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 生成一些虚拟数据
x_train = np.random.random((100, 10))
y_train = np.random.randint(2, size=(100, 1))

# 训练模型
model.fit(x_train, y_train, epochs=3, verbose=0)
print("   原始模型训练完成。")

# 2. 保存模型（两种格式）
print("2. 保存模型...")
model.save('my_saved_model')   # SavedModel 格式（目录）
model.save('my_model.h5')      # H5 格式（单个文件）
print("   模型已保存为两种格式。")

# 3. 加载模型
print("3. 加载模型...")
# 加载 SavedModel 格式
loaded_model_from_savedmodel = tf.keras.models.load_model('my_saved_model')

# 加载 H5 格式
loaded_model_from_h5 = tf.keras.models.load_model('my_model.h5')
print("   模型加载成功！")

# 4. 验证加载的模型
print("4. 验证加载的模型...")
# 生成一些测试数据
test_data = np.random.random((5, 10))

# 使用原始模型预测
original_pred = model.predict(test_data, verbose=0)
print(f"   原始模型预测结果: {original_pred.flatten()}")

# 使用加载的模型预测（结果应完全相同）
savedmodel_pred = loaded_model_from_savedmodel.predict(test_data, verbose=0)
h5_pred = loaded_model_from_h5.predict(test_data, verbose=0)

print(f"   SavedModel 预测结果: {savedmodel_pred.flatten()}")
print(f"   H5 模型预测结果:    {h5_pred.flatten()}")

# 验证预测结果是否相同 (使用 np.allclose 比较浮点数)
predictions_match = np.allclose(original_pred, savedmodel_pred) and np.allclose(original_pred, h5_pred)
print(f"   所有模型预测结果是否一致: {predictions_match}")

# 5. 检查模型功能
print("5. 检查模型功能完整性...")
print("   原始模型 summary:")
model.summary()

print("\n   加载的模型 (SavedModel) summary:")
loaded_model_from_savedmodel.summary()

# 尝试评估（需要 compile=True）
test_loss, test_acc = loaded_model_from_savedmodel.evaluate(x_train[:10], y_train[:10], verbose=0)
print(f"\n   加载的模型评估结果 - Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")

# 4. 处理自定义对象（关键！）
这是 `load_model()` 最重要的功能之一。当你的模型包含自定义组件时，必须提供 `custom_objects` 参数。

示例：加载包含自定义层的模型

In [ ]:
import tensorflow as tf

# 假设我们有一个自定义层
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                initializer='zeros',
                                trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {'units': self.units}

# 创建并保存包含自定义层的模型
model_with_custom = tf.keras.Sequential([
    CustomDenseLayer(64, input_shape=(10,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_with_custom.compile(optimizer='adam', loss='binary_crossentropy')
model_with_custom.save('custom_model') # 保存

# ！！！ 关键步骤：加载时提供 custom_objects ！！！
try:
    # 这样会报错：Unknown layer: CustomDenseLayer
    # loaded_model = tf.keras.models.load_model('custom_model')
    
    # 正确方式：提供自定义层的映射
    loaded_custom_model = tf.keras.models.load_model(
        'custom_model',
        custom_objects={'CustomDenseLayer': CustomDenseLayer} # 字典：{'层名': 层类}
    )
    print("包含自定义层的模型加载成功！")
    loaded_custom_model.summary()
except Exception as e:
    print(f"加载失败: {e}")

In [ ]:
示例：加载包含自定义损失函数的模型

In [ ]:
# 自定义损失函数
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred)) * 2.0

# 创建并使用自定义损失函数的模型
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(10,))])
model.compile(optimizer='adam', loss=custom_loss)
model.save('model_with_custom_loss.h5')

# 加载时提供自定义损失函数
loaded_model = tf.keras.models.load_model(
    'model_with_custom_loss.h5',
    custom_objects={'custom_loss': custom_loss} # 字典：{'函数名': 函数}
)
print("包含自定义损失函数的模型加载成功！")

# 5. 仅加载架构或权重
有时你可能只想加载模型架构或权重，而不是整个模型。

仅加载架构（从JSON）

In [ ]:
# 保存架构为JSON
json_string = model.to_json()

# 从JSON重建架构（不包含权重）
from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)

# 如果需要，可以随后加载权重
# model_architecture.load_weights('path_to_weights.h5')

In [ ]:
仅加载权重

In [ ]:
# 假设我们已经有一个结构完全相同的模型
new_model = tf.keras.Sequential([...]) # 结构与保存的模型相同
new_model.compile(...)

# 只加载权重
new_model.load_weights('path_to_weights.h5')
# 或者对于 SavedModel 格式的权重
# new_model.load_weights('my_saved_model/variables/variables')

# 6. 使用 compile=False 仅用于推理
如果你的目标只是使用模型进行预测，而不需要继续训练或评估，可以设置 `compile=False` 来加快加载速度并节省内存。

In [ ]:
# 快速加载模型用于生产环境预测
production_model = tf.keras.models.load_model(
    'my_saved_model',
    compile=False  # 不加载优化器、损失函数等训练配置
)

# 仍然可以完美地进行预测
predictions = production_model.predict(new_data)

# 但不能直接使用 .evaluate() 或 .fit()
# production_model.evaluate(...)  # 这会报错，因为模型没有编译